In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')


In [3]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [5]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])



In [6]:
from sklearn.compose import ColumnTransformer, make_column_transformer
onehotencoder= make_column_transformer(

(OneHotEncoder(categories='auto'), [1]),

remainder="passthrough")

X=onehotencoder.fit_transform(X)

In [8]:
X = X[:,1:]

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [14]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# ANN

In [17]:
import keras

In [18]:
from keras.models import Sequential

In [20]:
from keras.layers import Dense

# intializing ANN

In [21]:
classifier = Sequential()

# first layer

In [25]:
classifier.add(Dense( 6, kernel_initializer  = 'uniform', activation = 'relu', input_dim = 11))

# adding new layer

In [26]:
classifier.add(Dense( 6, kernel_initializer  = 'uniform', activation = 'relu'))

# output layer

In [27]:
classifier.add(Dense( 1, kernel_initializer  = 'uniform', activation = 'sigmoid'))

# compile ANN

In [28]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [29]:
classifier.fit(X_train,y_train, batch_size = 10, nb_epoch = 100 )

c:\users\ashish bisht\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/100
7500/7500 [==============================] - 2s 246us/step - loss: 0.4928 - accuracy: 0.7963
Epoch 2/100
7500/7500 [==============================] - 1s 172us/step - loss: 0.4246 - accuracy: 0.7963
Epoch 3/100
7500/7500 [==============================] - 1s 177us/step - loss: 0.4115 - accuracy: 0.8088
Epoch 4/100
7500/7500 [==============================] - 1s 173us/step - loss: 0.4006 - accuracy: 0.8297
Epoch 5/100
7500/7500 [==============================] - 1s 163us/step - loss: 0.3904 - accuracy: 0.8293
Epoch 6/100
7500/7500 [==============================] - 1s 171us/step - loss: 0.3820 - accuracy: 0.8349
Epoch 7/100
7500/7500 [==============================] - 1s 167us/step - loss: 0.3781 - accuracy: 0.8451
Epoch 8/100
7500/7500 [==============================] - 1s 163us/step - loss: 0.3735 - accuracy: 0.8479
Epoch 9/100
7500/7500 [==============================] - 1s 162us/step - loss: 0.3696 - accuracy: 0.8467
Epoch 10/100
7500/7500 [==============================]

In [30]:
y_pred = classifier.predict(X_test)

In [33]:
y_pred = (y_pred > .5)

In [34]:
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [35]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [36]:
cm

array([[1874,  117],
       [ 238,  271]], dtype=int64)

In [37]:
new_prediction = classifier.predict(sc.transform(np.array([[0,0,600,1,40,3,60000,2,1,1,50000]])))

In [38]:
new_prediction = (new_prediction > .5)

In [39]:
new_prediction

array([[False]])

# k-fold

In [42]:
from keras.wrappers.scikit_learn import KerasClassifier

In [43]:
from sklearn.model_selection import cross_val_score

In [44]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense( 6, kernel_initializer  = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense( 6, kernel_initializer  = 'uniform', activation = 'relu'))
    classifier.add(Dense( 1, kernel_initializer  = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies= cross_val_score(estimator = classifier, X = X_train, y = y_train, cv =10, n_jobs = -1)

In [46]:
accuracies.mean()

0.8386666655540467

In [47]:
accuracies.std()

0.01517893892184981

# how to apply dropout to overcome overfitting problems

from keras.layers import Dropout

classifier = Sequential()
classifier.add(Dense( 6, kernel_initializer  = 'uniform', activation = 'relu', input_dim = 11))
classifier.add(Dropout(p=.1))
classifier.add(Dense( 6, kernel_initializer  = 'uniform', activation = 'relu'))
classifier.add(Dropout(p=.1))
classifier.add(Dense( 1, kernel_initializer  = 'uniform', activation = 'sigmoid'))

# tuning ANN

In [49]:
from sklearn.model_selection import GridSearchCV